In [1]:
import numpy as np
import os
import re
import random
import pandas as pd
import json
import pickle
import platform
import sys
# 导入上级目录
sys.path.append('..')

from tqdm import tqdm

from mimic3models.preprocessing import Discretizer, Normalizer
#from mimic3models.keras_models.lstm import Network
#from mimic3benchmark.readers import InHospitalMortalityReader

In [2]:
dataset_dir='../mimic3benchmark/scripts/data/in-hospital-mortality'
train_listfile = os.path.join(dataset_dir, 'train_listfile.csv')
test_listfile = os.path.join(dataset_dir, 'test_listfile.csv')
result_dir = 'result/mortality' 
config_path = 'resources/discretizer_config.json'
output_dir = 'result'
mortify_outdir = os.path.join(output_dir, 'mortality')

partition = 'train'
timestep = 1.0
task = 'ihm'
impute_strategy = 'previous'
start_time = 'zero'
store_masks=True
deep_supervision = False

In [14]:
try:
    os.makedirs(result_dir)
except:
    pass

In [3]:
'''
在父类里面完成"partition"_listfile.csv读取操作
'''
class Reader(object):
    def __init__(self, dataset_dir='scripts/data/in-hospital-mortality', partition='train', listfile=None):
        self.dataset = dataset_dir
        self.current_index = 0
        self.partition = partition
        if listfile is None:
            listfile_path = os.path.join(dataset_dir, self.partition+'_listfile.csv')
        else:
            listfile_path = listfile
        # 获取"partition"_listfile.csv的内容
        # 形式： *.csv, y_true
        self.data = pd.read_csv(listfile_path)
        self.data['y_true'] = self.data['y_true'].astype('int')
        self.header = self.data.columns.values.tolist()
    def get_number_of_examples(self):
        return self.data.shape[0]
    def random_shuffle(self, seed=None):
        # 如果是训练集，打乱数据
        if self.partition == 'train':
            self.data = self.data.sample(frac=1.0)
    #  reads the sample with the given index
    def read_sample(self, index):
        raise NotImplementedError()
    #  read_next reads the next sample by using a cyclic counter inside
    def read_next(self):
        to_read_index = self.current_index
        self.current_index += 1
        if self.current_index == self.get_number_of_examples():
            self.current_index = 0
        return self.read_sample(to_read_index)
    

In [4]:
""" Reader for in-hospital moratality prediction task.

:param dataset_dir:   Directory where timeseries files are stored.
:param listfile:      Path to a listfile. If this parameter is left `None` then
                        `dataset_dir/listfile.csv` will be used.
:param period_length: Length of the period (in hours) from which the prediction is done.
"""
class InHospitalMortalityReader(Reader):
    def __init__(self, dataset_dir='scripts/data/in-hospital-mortality', partition='train', period_length=48.0, listfile=None):
        Reader.__init__(self, dataset_dir, listfile=None)
        # self.data是父类读取的"partition"_listfile.csv的内容
        self.data_x = self.data['patient_id_i'].values.tolist()
        self.data_y = self.data['y_true'].values.tolist()
        self.period_length = period_length
        #print(self.data_x)
    # 读取self.data_x的csv文件，提取里面的17个变量信息
    def read_timeseries(self, ts_filename):
        ret = []
        timeseries = pd.read_csv(ts_filename)
        timeseries = timeseries.fillna('')
        header = timeseries.columns.values.tolist()
        assert header[0] == 'Hours'
        for index, row in timeseries.iterrows():
            variables = [row[header[i]] for i in range(len(header))]
            ret.append(np.array(variables))
        return (np.stack(ret), header)
    def read_example(self, index):
        """ 
        Read the example with given index.

            :param index: Index of the line of the listfile to read (counting starts from 0).
            :return: Directory with the following keys:
                X : np.array
                    2D array containing all events. Each row corresponds to a moment.
                    First column is the time and other columns correspond to different
                    variables.
                t : float
                    Length of the data in hours. Note, in general, it is not equal to the
                    timestamp of last event.
                y : int (0 or 1)
                    Mortality within next 24 hours.
                header : array of strings
                    Names of the columns. The ordering of the columns is always the same.
                name: Name of the sample.
        """
        if index < 0 or index >= len(self.data):
            return ValueError("Index must be from 0 (inclusive) to number of examples (exclusive).")
        name = self.data_x[index]
        name_path = os.path.join(dataset_dir, partition, name)
        y = self.data_y[index]
        t = self.period_length
        (X, header) = self.read_timeseries(name_path)
        return {
            'X': X,
            't': t,
            'y': y,
            'header': header,
            'name': name
        }



### Computes means and standard variations on the train data and uses those statistics to normalize both the train and test data

In [5]:
train_reader = InHospitalMortalityReader(dataset_dir=dataset_dir, partition='train', listfile=os.path.join(output_dir, 'mortality', 'train_listfile.csv'))
val_reader = InHospitalMortalityReader(dataset_dir=dataset_dir, partition='train', listfile=os.path.join(output_dir, 'mortality', 'val_listfile.csv'))

n_samples = train_reader.get_number_of_examples()
# create the discretizer
'''
a dataset example: 
return {
            'X': X,
            't': t,
            'y': y,
            'header': header,
            'name': name
        }
'''
# 离散变量标准化的类
discretizer = Discretizer(timestep=timestep,
                            store_masks=store_masks,
                            impute_strategy=impute_strategy,
                            start_time=start_time)
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
# 连续变量在discretizer_header中的index
# discretizer是对离散变量进行onehot标准化； cont_channels代表连续变量，discretizer并未对其进行处理
continuous_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]
#print(continuous_channels)
normalizer = Normalizer(fields=continuous_channels) # choose here which columns to standardize

#### 得到连续变量的mean和std

In [7]:
""" for i in tqdm(range(n_samples), desc='Iterating over train patients\' icu stays'):
    ret = train_reader.read_example(i)
    #print(ret)
    data, new_header = discretizer.transform(ret['X'], end=ret['t'])
    #print(data[0])
    normalizer._feed_data(data)

file_name = '{}_ts_{:.2f}_impute_{}_start_{}_masks_{}_n_{}_normalizer.pickle'.format(
       task, timestep, impute_strategy, start_time, store_masks, n_samples)
file_name = os.path.join(output_dir, file_name)
print('\n Saving the state in {} ...'.format(file_name))
normalizer._save_params(file_name)
print('saved!') """

Iterating over train patients' icu stays: 100%|██████████| 14803/14803 [11:45<00:00, 20.99it/s]
 Saving the state in result\ihm_ts_1.00_impute_previous_start_zero_masks_True_n_14803_normalizer.pickle ...
saved!



#### 载入连续变量的mean和std

In [6]:
normalizer_state = '{}_ts_{:.2f}_impute_{}_start_{}_masks_{}_n_{}_normalizer.pickle'.format(
       task, timestep, impute_strategy, start_time, store_masks, n_samples)
normalizer.load_params(os.path.join(output_dir, normalizer_state))
#print(normalizer._means)

# Build Model

In [7]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  Dense, LSTM, Masking, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

In [8]:
network = 'mimic3models/keras_models/lstm.py'
# Build the model
print("==> using model: {}".format(network))


==> using model: mimic3models/keras_models/lstm.py


In [9]:
class IHMNetwork(Model):
    def __init__(self, dim=16, batch_size=8, dropout=0.3, task='ihm',  num_classes=1, depth=2, input_dim=76, is_bidirectional=True):
        self.dim = dim
        self.batch_size = batch_size
        self.dropout = dropout
        self.task = task
        self.num_classes = num_classes
        self.depth = depth
        self.input_dim = input_dim
        self.is_bidirectional = True
        self.final_activation = 'sigmoid'
        self.return_sequences = False
        
        """ if self.task in ['decomp', 'ihm', 'ph']:
            self.final_activation = 'sigmoid'
        elif task in ['los']:
            if num_classes == 1:
                self.final_activation = 'relu'
            else:
                self.final_activation = 'softmax' """

        # input layer 
        # input shape: [batch * input_dim]
        X = Input(shape=(None, self.input_dim), name='X')
        inputs = [X]
        mX = Masking()(X)

        for i in range(self.depth - 1):
            num_units = self.dim
            if self.is_bidirectional:
                num_units = num_units // 2

            lstm = LSTM(units=num_units,
                        activation='tanh',
                        return_sequences=True,
                        recurrent_dropout=0,
                        dropout=self.dropout)

            if is_bidirectional:
                mX = Bidirectional(lstm)(mX)
            else:
                mX = lstm(mX)

        L = LSTM(units=self.dim,
                 activation='tanh',
                 return_sequences=self.return_sequences,
                 dropout=self.dropout,
                 recurrent_dropout=0)(mX)
        if self.dropout > 0:
            L = Dropout(self.dropout)(L)

        y = Dense(self.num_classes, activation=self.final_activation)(L)
        outputs = [y]

        super(IHMNetwork, self).__init__(inputs=inputs, outputs=outputs)



            








In [10]:
# create ihm model
ihm_model = IHMNetwork()
# Compile the model
print("==> compiling the model")
optimizer = 'adam'
loss = 'binary_crossentropy'
loss_weights = None
ihm_model.compile(optimizer=optimizer,
              loss=loss,
              loss_weights=loss_weights)
ihm_model.summary()

==> compiling the model
Model: "ihm_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               [(None, None, 76)]        0         
_________________________________________________________________
masking (Masking)            (None, None, 76)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 16)          5440      
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 7,569
Trainable params: 7,569
Non-trainable params: 0
_____________________________

# load data

In [11]:
def read_chunk(reader, chunk_size):
    data = {}
    for i in range(chunk_size):
        ret = reader.read_example(i)
        for k, v in ret.items():
            if k not in data:
                data[k] = []
            data[k].append(v)
    data["header"] = data["header"][0]
    return data


In [12]:
def load_data(reader, discretizer, normalizer, small_part=False, return_names=False):
    if small_part:
        N = 1000
    else:
        N = reader.get_number_of_examples()

    ret = read_chunk(reader, N)
    data = ret["X"]
    ts = ret["t"]
    labels = ret["y"]
    names = ret["name"]
    data = [discretizer.transform(X, end=t)[0] for (X, t) in zip(data, ts)]
    if normalizer is not None:
        data = [normalizer.transform(X) for X in data]
    whole_data = (np.array(data), labels)
    if not return_names:
        return whole_data
    return {"data": whole_data, "names": names}

In [13]:
# Read data
# 先使用小规模的数据
small_part=True
train_raw =load_data(train_reader, discretizer, normalizer, small_part)
val_raw = load_data(val_reader, discretizer, normalizer, small_part)

ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 63. ...  0.  0.  0.]
 [ 1.  0. 67. ...  0.  0.  0.]
 [ 1.  0. 67. ...  0.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 64. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  1.]
 [ 1.  0. 61. ...  1.  0.  1.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 51. ...  1.  0.  0.]
 ...
 [ 1.  0. 54. ...  0.  0.  0.]
 [ 1.  0. 51. ...  0.  0.  0.]
 [ 1.  0. 58. ...  1.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 39. ...  0.  0.  0.]
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 38. ...  0.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 65. ...  0.  0.  0.]]
ret: [[ 1.  0. 

In [14]:
val_raw = load_data(val_reader, discretizer, normalizer, small_part)

ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 63. ...  0.  0.  0.]
 [ 1.  0. 67. ...  0.  0.  0.]
 [ 1.  0. 67. ...  0.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 64. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  1.]
 [ 1.  0. 61. ...  1.  0.  1.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 51. ...  1.  0.  0.]
 ...
 [ 1.  0. 54. ...  0.  0.  0.]
 [ 1.  0. 51. ...  0.  0.  0.]
 [ 1.  0. 58. ...  1.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 39. ...  0.  0.  0.]
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 38. ...  0.  0.  0.]]
ret: [[ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 [ 1.  0. 59. ...  0.  0.  0.]
 ...
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 53. ...  0.  0.  0.]
 [ 1.  0. 65. ...  0.  0.  0.]]
ret: [[ 1.  0. 

In [16]:
print(len(train_raw))
# train_raw 里面分别是X和label的内容
# X shape : 1000*48*76 ,其中1000是指定读取前1000个病人的数据作为示例
print(train_raw[0].shape)
print(len(train_raw[1]))

print(val_raw[0].shape)
print(len(val_raw[1]))
print(type(train_raw))

2
(1000, 48, 76)
1000
(1000, 48, 76)
1000


In [37]:
from mimic3models import metrics
class InHospitalMortalityMetrics(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, batch_size=32, early_stopping=True, verbose=2):
        super(InHospitalMortalityMetrics, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.batch_size = batch_size
        self.early_stopping = early_stopping
        self.verbose = verbose
        self.train_history = []
        self.val_history = []

    def calc_metrics(self, data, history, dataset, logs):
        y_true = []
        predictions = []
        B = self.batch_size
        for i in range(0, len(data[0]), B):
            if self.verbose == 1:
                print("\tdone {}/{}".format(i, len(data[0])), end='\r')
            (x, y) = (data[0][i:i + B], data[1][i:i + B])
            outputs = self.model.predict(x, batch_size=B)
            predictions += list(np.array(outputs).flatten())
            y_true += list(np.array(y).flatten())
        print('\n')
        predictions = np.array(predictions)
        predictions = np.stack([1 - predictions, predictions], axis=1)
        ret = metrics.print_metrics_binary(y_true, predictions)
        for k, v in ret.items():
            logs[dataset + '_' + k] = v
        history.append(ret)

    def on_epoch_end(self, epoch, logs={}):
        print("\n==>predicting on train")
        self.calc_metrics(self.train_data, self.train_history, 'train', logs)
        #print("\n==>predicting on validation")
        #self.calc_metrics(self.val_data, self.val_history, 'val', logs)

        

In [38]:
#path = os.path.join(output_dir, 'keras_states/' + 'ihm_lstm' + '_epoch{epoch}_test{val_loss}_state')
epochs = 2
# epoch = 28
batch_size = 8
save_every = 2 ** 8
verbose = 1
path = os.path.join(output_dir, 'mortality', 'keras_states' , 'ihm_lstm' + '_epoch{epoch}_state')
metrics_callback = InHospitalMortalityMetrics(train_data=train_raw,
                                                            val_data=val_raw,
                                                            batch_size=batch_size,
                                                            verbose=verbose)
# make sure save directory exists
dirname = os.path.dirname(path)
if not os.path.exists(dirname):
    os.makedirs(dirname)
saver = ModelCheckpoint(path, verbose=1, period= save_every)

keras_logs = os.path.join(output_dir, 'mortality' ,'keras_logs')
if not os.path.exists(keras_logs):
    os.makedirs(keras_logs)
csv_logger = CSVLogger(os.path.join(keras_logs, 'mortality_lstm.csv'),
                        append=True, separator=';')

print("==> training")
ihm_model.fit(x=np.array(train_raw[0]),
            y=np.array(train_raw[1]),
            #validation_data=val_raw,
            epochs=  epochs,
            callbacks=[metrics_callback, saver, csv_logger],
            shuffle=True,
            verbose=1,
            batch_size=batch_size)

==> training
Epoch 1/2
125/125 [==============================] - ETA: 0s - loss: 0.3059
==>predicting on train
	done 992/1000

confusion matrix:
[[840  23]
 [ 94  43]]
accuracy = 0.8830000162124634
precision class 0 = 0.8993576169013977
precision class 1 = 0.6515151262283325
recall class 0 = 0.9733487963676453
recall class 1 = 0.31386861205101013
AUC of ROC = 0.8907393154079726
AUC of PRC = 0.6065066359144076
min(+P, Se) = 0.5664335664335665
125/125 [==============================] - 18s 144ms/step - loss: 0.3059
Epoch 2/2
125/125 [==============================] - ETA: 0s - loss: 0.2876
==>predicting on train


confusion matrix:
[[839  24]
 [ 91  46]]
accuracy = 0.8849999904632568
precision class 0 = 0.9021505117416382
precision class 1 = 0.6571428775787354
recall class 0 = 0.9721900224685669
recall class 1 = 0.33576643466949463
AUC of ROC = 0.892447835170133
AUC of PRC = 0.6161043888647169
min(+P, Se) = 0.6014492753623188
125/125 [==============================] - 18s 140ms/step - l

# tries

In [96]:
""" exam = pd.read_csv(os.path.join(dataset_dir, 'train', '11093_1.csv'))
print(exam.dtypes)
X = train_reader.read_example(0)['X']
print(X)
ts = [float(row[0]) for row in X]
first_time = ts[0]
max_hours = max(ts) - first_time
print(max_hours)
eps = 1e-6
N_bins = int(max_hours / 0.8 + 1.0 - eps)
print(N_bins) """

In [39]:
 with open(config_path) as f:
     config = json.load(f)

id_to_channel = config['id_to_channel']
print(f'id_to_channel: {id_to_channel}')
channel_to_id = dict(zip(id_to_channel, range(len(id_to_channel))))
print(f'channel_to_id: {channel_to_id}')
is_categorical_channel = config['is_categorical_channel']
possible_values = config['possible_values']
normal_values = config['normal_values']
# 设置header： hours + 17个变量
header = ["Hours"] + id_to_channel
timestep = 0.8
# for statistics
done_count = 0
empty_bins_sum = 0
unused_data_sum = 0

id_to_channel: ['Capillary refill rate', 'Diastolic blood pressure', 'Fraction inspired oxygen', 'Glascow coma scale eye opening', 'Glascow coma scale motor response', 'Glascow coma scale total', 'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure', 'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH']
channel_to_id: {'Capillary refill rate': 0, 'Diastolic blood pressure': 1, 'Fraction inspired oxygen': 2, 'Glascow coma scale eye opening': 3, 'Glascow coma scale motor response': 4, 'Glascow coma scale total': 5, 'Glascow coma scale verbal response': 6, 'Glucose': 7, 'Heart Rate': 8, 'Height': 9, 'Mean blood pressure': 10, 'Oxygen saturation': 11, 'Respiratory rate': 12, 'Systolic blood pressure': 13, 'Temperature': 14, 'Weight': 15, 'pH': 16}


In [108]:
N_channels = len(id_to_channel)
begin_pos = [0 for i in range(N_channels)]
end_pos = [0 for i in range(N_channels)]
cur_len = 0
for i in range(N_channels):
    # channel： 当前变量的名称
    channel = id_to_channel[i]
    begin_pos[i] = cur_len
    # 如果该变量属于类别变量（只有类别变量才会有可能的取值）
    if is_categorical_channel[channel]:
        end_pos[i] = begin_pos[i] + len(possible_values[channel])
    else:
        end_pos[i] = begin_pos[i] + 1
    cur_len = end_pos[i]

    # 上面的这个begin_pos数组和end_pos数组记录的是每个变量所包含的可能的内容的开始和结束的下标
data = np.zeros(shape=(N_bins, cur_len), dtype=float)
mask = np.zeros(shape=(N_bins, N_channels), dtype=int)
original_value = [["" for j in range(N_channels)] for i in range(N_bins)]
total_data = 0
unused_data = 0

def write(data, bin_id, channel, value, begin_pos):
    channel_id = channel_to_id[channel]
    # 如果是类别数据，找到该value对应的类别的index
    if is_categorical_channel[channel]:
        #print(channel)
        if channel=='Glascow coma scale total': 
            #print(type(value))
            value = value.astype(float)
            #print(type(value))
            value = value.astype(int)
            value = value.astype(str)
        #print(f'value: {value}')
        category_id = possible_values[channel].index(value)
        # 该数据有多少可能的取值
        N_values = len(possible_values[channel])
        one_hot = np.zeros((N_values,))
        one_hot[category_id] = 1
        for pos in range(N_values):
            # 这个地方扩充了X的长度
            # 由 17 ->  17+类别数据的所有可能取值数
            data[bin_id, begin_pos[channel_id] + pos] = one_hot[pos]
    #  不是类别数据直接就记录value
    else:
        data[bin_id, begin_pos[channel_id]] = float(value)

for index, row in enumerate(X):
    print(f'index: {index}')
    t = float(row[0]) - first_time
    if t > max_hours + eps:
        continue
    # 重新归置每一个指标的测量时间
    # 即： 给每一个指标的测量时间做一个全体平移
    bin_id = int(t / timestep - eps)
    #print(f'hours: {row[0]}; bin_id: {bin_id}')
    assert 0 <= bin_id < N_bins
    # 遍历这17个变量的测量值
    for j in range(1, len(row)):
        if row[j] == "":
            continue
        print(row[j])
        #print(type(row[j]))

        # 找到不为空的变量
        # 找到其对应的指标名称
        channel = header[j]
        channel_id = channel_to_id[channel]

        total_data += 1
        if mask[bin_id][channel_id] == 1:
            unused_data += 1
        mask[bin_id][channel_id] = 1
        
        original_value[bin_id][channel_id] = row[j]
        write(data, bin_id, channel, row[j], begin_pos)
        #original_value[bin_id][channel_id] = row[j]

index: 0
7.43
index: 1
128.0
6.5
index: 2
7.0
index: 3
61.0
56.0
76.0
100.0
16.0
106.0
37.94444613986545
45.0
index: 4
61.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
72.0
76.0
100.0
23.0
106.0
index: 5
54.0
89.0
72.66670227050781
100.0
22.0
110.0
index: 6
56.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
90.0
77.33329772949219
100.0
4.0
120.0
37.77780151367188
index: 7
47.0
73.0
69.33329772949217
100.0
21.0
114.0
index: 8
62.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
65.0
87.0
100.0
18.0
137.0
index: 9
112.0
index: 10
7.46
index: 11
71.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
68.0
94.0
100.0
15.0
139.0
index: 12
79.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
80.0
104.0
100.0
21.0
154.0
38.27780151367188
index: 13
81.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
91.0
105.0
100.0
22.0
148.0
index: 14
78.0
1 No Response
4 Flex-withdraws
6.0
1.0 ET/Trach
92.0
95.0
100.0
22.0
143.0
38.16669845581055
index: 15
75.0
1 No Response
4 Flex-withdraws
6.0
1 No Re

# impute holes

In [ ]:
impute_strategy = "zero"


In [ ]:
discretizer = Discretizer(timestep=float(args.timestep),
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = args.normalizer_state
if normalizer_state is None:
    normalizer_state = 'ihm_ts{}.input_str:{}.start_time:zero.normalizer'.format(args.timestep, args.imputation)
    normalizer_state = os.path.join(os.path.dirname(__file__), normalizer_state)
normalizer.load_params(normalizer_state)

args_dict = dict(args._get_kwargs())
args_dict['header'] = discretizer_header
args_dict['task'] = 'ihm'
args_dict['target_repl'] = target_repl

# Build the model
print("==> using model {}".format(args.network))
model_module = imp.load_source(os.path.basename(args.network), args.network)
model = model_module.Network(**args_dict)
suffix = ".bs{}{}{}.ts{}{}".format(args.batch_size,
                                   ".L1{}".format(args.l1) if args.l1 > 0 else "",
                                   ".L2{}".format(args.l2) if args.l2 > 0 else "",
                                   args.timestep,
                                   ".trc{}".format(args.target_repl_coef) if args.target_repl_coef > 0 else "")
model.final_name = args.prefix + model.say_name() + suffix
print("==> model.final_name:", model.final_name)


# Compile the model
print("==> compiling the model")
optimizer_config = {'class_name': args.optimizer,
                    'config': {'lr': args.lr,
                               'beta_1': args.beta_1}}

# NOTE: one can use binary_crossentropy even for (B, T, C) shape.
#       It will calculate binary_crossentropies for each class
#       and then take the mean over axis=-1. Tre results is (B, T).
if target_repl:
    loss = ['binary_crossentropy'] * 2
    loss_weights = [1 - args.target_repl_coef, args.target_repl_coef]
else:
    loss = 'binary_crossentropy'
    loss_weights = None

model.compile(optimizer=optimizer_config,
              loss=loss,
              loss_weights=loss_weights)
model.summary()

# Load model weights
n_trained_chunks = 0
if args.load_state != "":
    model.load_weights(args.load_state)
    n_trained_chunks = int(re.match(".*epoch([0-9]+).*", args.load_state).group(1))


# Read data
train_raw = utils.load_data(train_reader, discretizer, normalizer, args.small_part)
val_raw = utils.load_data(val_reader, discretizer, normalizer, args.small_part)

if target_repl:
    T = train_raw[0][0].shape[0]

    def extend_labels(data):
        data = list(data)
        labels = np.array(data[1])  # (B,)
        data[1] = [labels, None]
        data[1][1] = np.expand_dims(labels, axis=-1).repeat(T, axis=1)  # (B, T)
        data[1][1] = np.expand_dims(data[1][1], axis=-1)  # (B, T, 1)
        return data

    train_raw = extend_labels(train_raw)
    val_raw = extend_labels(val_raw)

if args.mode == 'train':

    # Prepare training
    path = os.path.join(args.output_dir, 'keras_states/' + model.final_name + '.epoch{epoch}.test{val_loss}.state')

    metrics_callback = keras_utils.InHospitalMortalityMetrics(train_data=train_raw,
                                                              val_data=val_raw,
                                                              target_repl=(args.target_repl_coef > 0),
                                                              batch_size=args.batch_size,
                                                              verbose=args.verbose)
    # make sure save directory exists
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    saver = ModelCheckpoint(path, verbose=1, period=args.save_every)

    keras_logs = os.path.join(args.output_dir, 'keras_logs')
    if not os.path.exists(keras_logs):
        os.makedirs(keras_logs)
    csv_logger = CSVLogger(os.path.join(keras_logs, model.final_name + '.csv'),
                           append=True, separator=';')

    print("==> training")
    model.fit(x=train_raw[0],
              y=train_raw[1],
              validation_data=val_raw,
              epochs=n_trained_chunks + args.epochs,
              initial_epoch=n_trained_chunks,
              callbacks=[metrics_callback, saver, csv_logger],
              shuffle=True,
              verbose=args.verbose,
              batch_size=args.batch_size)

elif args.mode == 'test':

    # ensure that the code uses test_reader
    del train_reader
    del val_reader
    del train_raw
    del val_raw

    test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(args.data, 'test'),
                                            listfile=os.path.join(args.data, 'test_listfile.csv'),
                                            period_length=48.0)
    ret = utils.load_data(test_reader, discretizer, normalizer, args.small_part,
                          return_names=True)

    data = ret["data"][0]
    labels = ret["data"][1]
    names = ret["names"]

    predictions = model.predict(data, batch_size=args.batch_size, verbose=1)
    predictions = np.array(predictions)[:, 0]
    metrics.print_metrics_binary(labels, predictions)

    path = os.path.join(args.output_dir, "test_predictions", os.path.basename(args.load_state)) + ".csv"
    utils.save_results(names, predictions, labels, path)

else:
    raise ValueError("Wrong value for args.mode")


In [ ]:
import numpy as np
import platform
import pickle
import json
import os


class Discretizer:
    def __init__(self, timestep=0.8, store_masks=True, impute_strategy='zero', start_time='zero',
                 config_path=os.path.join(os.path.dirname(__file__), 'resources/discretizer_config.json')):

        with open(config_path) as f:
            config = json.load(f)
            self._id_to_channel = config['id_to_channel']
            self._channel_to_id = dict(zip(self._id_to_channel, range(len(self._id_to_channel))))
            self._is_categorical_channel = config['is_categorical_channel']
            self._possible_values = config['possible_values']
            self._normal_values = config['normal_values']

        self._header = ["Hours"] + self._id_to_channel
        self._timestep = timestep
        self._store_masks = store_masks
        self._start_time = start_time
        self._impute_strategy = impute_strategy

        # for statistics
        self._done_count = 0
        self._empty_bins_sum = 0
        self._unused_data_sum = 0

    def transform(self, X, header=None, end=None):
        if header is None:
            header = self._header
        assert header[0] == "Hours"
        eps = 1e-6

        N_channels = len(self._id_to_channel)
        ts = [float(row[0]) for row in X]
        for i in range(len(ts) - 1):
            assert ts[i] < ts[i+1] + eps

        if self._start_time == 'relative':
            first_time = ts[0]
        elif self._start_time == 'zero':
            first_time = 0
        else:
            raise ValueError("start_time is invalid")

        if end is None:
            max_hours = max(ts) - first_time
        else:
            max_hours = end - first_time

        N_bins = int(max_hours / self._timestep + 1.0 - eps)

        cur_len = 0
        begin_pos = [0 for i in range(N_channels)]
        end_pos = [0 for i in range(N_channels)]
        for i in range(N_channels):
            channel = self._id_to_channel[i]
            begin_pos[i] = cur_len
            if self._is_categorical_channel[channel]:
                end_pos[i] = begin_pos[i] + len(self._possible_values[channel])
            else:
                end_pos[i] = begin_pos[i] + 1
            cur_len = end_pos[i]

        data = np.zeros(shape=(N_bins, cur_len), dtype=float)
        mask = np.zeros(shape=(N_bins, N_channels), dtype=int)
        original_value = [["" for j in range(N_channels)] for i in range(N_bins)]
        total_data = 0
        unused_data = 0

        def write(data, bin_id, channel, value, begin_pos):
            channel_id = self._channel_to_id[channel]
            if self._is_categorical_channel[channel]:
                category_id = self._possible_values[channel].index(value)
                N_values = len(self._possible_values[channel])
                one_hot = np.zeros((N_values,))
                one_hot[category_id] = 1
                for pos in range(N_values):
                    data[bin_id, begin_pos[channel_id] + pos] = one_hot[pos]
            else:
                data[bin_id, begin_pos[channel_id]] = float(value)

        for row in X:
            t = float(row[0]) - first_time
            if t > max_hours + eps:
                continue
            bin_id = int(t / self._timestep - eps)
            assert 0 <= bin_id < N_bins

            for j in range(1, len(row)):
                if row[j] == "":
                    continue
                channel = header[j]
                channel_id = self._channel_to_id[channel]

                total_data += 1
                if mask[bin_id][channel_id] == 1:
                    unused_data += 1
                mask[bin_id][channel_id] = 1

                write(data, bin_id, channel, row[j], begin_pos)
                original_value[bin_id][channel_id] = row[j]

        # impute missing values

        if self._impute_strategy not in ['zero', 'normal_value', 'previous', 'next']:
            raise ValueError("impute strategy is invalid")

        if self._impute_strategy in ['normal_value', 'previous']:
            prev_values = [[] for i in range(len(self._id_to_channel))]
            for bin_id in range(N_bins):
                for channel in self._id_to_channel:
                    channel_id = self._channel_to_id[channel]
                    if mask[bin_id][channel_id] == 1:
                        prev_values[channel_id].append(original_value[bin_id][channel_id])
                        continue
                    if self._impute_strategy == 'normal_value':
                        imputed_value = self._normal_values[channel]
                    if self._impute_strategy == 'previous':
                        if len(prev_values[channel_id]) == 0:
                            imputed_value = self._normal_values[channel]
                        else:
                            imputed_value = prev_values[channel_id][-1]
                    write(data, bin_id, channel, imputed_value, begin_pos)

        if self._impute_strategy == 'next':
            prev_values = [[] for i in range(len(self._id_to_channel))]
            for bin_id in range(N_bins-1, -1, -1):
                for channel in self._id_to_channel:
                    channel_id = self._channel_to_id[channel]
                    if mask[bin_id][channel_id] == 1:
                        prev_values[channel_id].append(original_value[bin_id][channel_id])
                        continue
                    if len(prev_values[channel_id]) == 0:
                        imputed_value = self._normal_values[channel]
                    else:
                        imputed_value = prev_values[channel_id][-1]
                    write(data, bin_id, channel, imputed_value, begin_pos)

        empty_bins = np.sum([1 - min(1, np.sum(mask[i, :])) for i in range(N_bins)])
        self._done_count += 1
        self._empty_bins_sum += empty_bins / (N_bins + eps)
        self._unused_data_sum += unused_data / (total_data + eps)

        if self._store_masks:
            data = np.hstack([data, mask.astype(np.float32)])

        # create new header
        new_header = []
        for channel in self._id_to_channel:
            if self._is_categorical_channel[channel]:
                values = self._possible_values[channel]
                for value in values:
                    new_header.append(channel + "->" + value)
            else:
                new_header.append(channel)

        if self._store_masks:
            for i in range(len(self._id_to_channel)):
                channel = self._id_to_channel[i]
                new_header.append("mask->" + channel)

        new_header = ",".join(new_header)

        return (data, new_header)

    def print_statistics(self):
        print("statistics of discretizer:")
        print("\tconverted {} examples".format(self._done_count))
        print("\taverage unused data = {:.2f} percent".format(100.0 * self._unused_data_sum / self._done_count))
        print("\taverage empty  bins = {:.2f} percent".format(100.0 * self._empty_bins_sum / self._done_count))

